# Model Merging with SparseGPT on Kaggle

This notebook demonstrates how to merge fine-tuned LLaMA models using three methods:
1. TIES (magnitude-based)
2. DARE (random dropout)
3. SparseGPT (Hessian-based importance)

**Prerequisites:**
- Enable GPU (Settings → Accelerator → GPU T4 x2)
- Enable Internet (Settings → Internet → On)
- Add your fine-tuned models as Kaggle datasets
- ⚠️ **NO HuggingFace account needed!** (unless using private models)

## Step 1: Setup Environment

In [ ]:
# Run setup script
!python kaggle_setup.py

In [ ]:
# Copy models from Kaggle input to working directory
# Update the paths below to match your Kaggle dataset names

import shutil
import os

# Example: Copy your fine-tuned model(s)
kaggle_datasets = [
    "/kaggle/input/llama3-2-1b-instruct-ft-doctor-consulting",
    # Add more dataset paths here if you have multiple models
]

local_models = []

for i, dataset_path in enumerate(kaggle_datasets):
    if os.path.exists(dataset_path):
        local_path = f"./finetuned_model_{i+1}"
        print(f"Copying {dataset_path} → {local_path}")
        shutil.copytree(dataset_path, local_path)
        
        # Verify files
        files = os.listdir(local_path)
        print(f"  ✓ Copied {len(files)} files")
        
        local_models.append(local_path)
    else:
        print(f"  ⚠️ Dataset not found: {dataset_path}")

print(f"\n✓ Ready to use {len(local_models)} model(s)")
print(f"Local paths: {local_models}")

## Step 1A: Copy Models from Kaggle Datasets (If Using Kaggle Datasets)

**IMPORTANT:** If your fine-tuned models are uploaded as Kaggle datasets, copy them first!

## Step 2: Configure HuggingFace Token (OPTIONAL - Only if using HF models)

**Skip this step if you copied models from Kaggle datasets above!**

Only needed if:
- Using models from HuggingFace Hub
- Using private HuggingFace models
- Base model requires authentication

In [ ]:
import os
from huggingface_hub import login

# Option 1: Use Kaggle Secrets (recommended)
# Add HF_TOKEN in Kaggle Secrets
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN")

# Option 2: Direct token (not recommended for public notebooks)
# hf_token = "hf_..."

login(token=hf_token)
print("✓ Logged in to HuggingFace")

## Step 3: Configure Merging Parameters

In [ ]:
import torch
from llama_merge import LLaMAMerger

# Configuration
BASE_MODEL = "meta-llama/Llama-3.2-1B-Instruct"

# OPTION A: Using models copied from Kaggle datasets (from Step 1A)
# Use the local_models list created above
FINETUNED_MODELS = local_models  # ["./finetuned_model_1", "./finetuned_model_2", ...]

# OPTION B: Using HuggingFace model IDs (requires Step 2)
# FINETUNED_MODELS = [
#     "your-username/model-name",
#     "your-username/another-model",
# ]

# Calibration datasets (one per model)
# These can be HuggingFace dataset names
DATASETS = [
    "Amod/mental_health_counseling_conversations",
    # Add more datasets here (one per fine-tuned model)
]

# Make sure you have same number of datasets as models
if len(DATASETS) < len(FINETUNED_MODELS):
    # Duplicate the first dataset if needed
    DATASETS = DATASETS * len(FINETUNED_MODELS)
    DATASETS = DATASETS[:len(FINETUNED_MODELS)]

# Parameters
OUTPUT_DIR = "./merged_models"
CACHE_DIR = "./merge_cache"
DENSITY = 0.2  # Keep top 20% of weights
NUM_CALIBRATION_SAMPLES = 128
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Device: {DEVICE}")
print(f"Base model: {BASE_MODEL}")
print(f"Fine-tuned models: {FINETUNED_MODELS}")
print(f"Datasets: {DATASETS}")
print(f"Density: {DENSITY}")

## Step 4: Initialize Merger

In [ ]:
# Create merger instance
merger = LLaMAMerger(
    base_model_path=BASE_MODEL,
    finetuned_model_paths=FINETUNED_MODELS,
    dataset_names=DATASETS,
    output_dir=OUTPUT_DIR,
    cache_dir=CACHE_DIR,
    density=DENSITY,
    num_calibration_samples=NUM_CALIBRATION_SAMPLES,
    device=DEVICE
)

print("✓ Merger initialized")

## Step 5: Run All Methods and Compare

In [ ]:
# This will:
# 1. Compute task vectors (if not cached)
# 2. Compute Hessians (if not cached)
# 3. Merge with all three methods
# 4. Evaluate and compare

results = merger.merge_all_methods()

print("\n" + "="*80)
print("RESULTS SUMMARY")
print("="*80)
for method, metrics in results.items():
    print(f"\n{method}:")
    print(f"  Perplexity: {metrics['perplexity']:.4f}")
    print(f"  Time: {metrics['time']:.2f}s")

# Find best method
best_method = min(results, key=lambda k: results[k]['perplexity'])
print(f"\n🏆 Best method: {best_method}")
print(f"   Perplexity: {results[best_method]['perplexity']:.4f}")

## Step 6: Save Best Model

In [ ]:
# The models are already saved in OUTPUT_DIR
# You can load and test them:

from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the best model (usually TIES-SparseGPT)
best_model_path = f"{OUTPUT_DIR}/ties_sparsegpt_merged"

model = AutoModelForCausalLM.from_pretrained(best_model_path)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

print(f"✓ Loaded best model from {best_model_path}")

## Step 7: Test the Merged Model

In [ ]:
# Test with a sample prompt
prompt = "I've been feeling anxious lately. What should I do?"

inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)
outputs = model.generate(
    **inputs,
    max_new_tokens=200,
    temperature=0.7,
    do_sample=True,
    top_p=0.9
)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Prompt:", prompt)
print("\nResponse:", response)

## Optional: Run Individual Methods

If you want to run just one method:

In [ ]:
# Run only SparseGPT method
sparsegpt_model = merger.merge_with_ties(use_sparsegpt=True)
sparsegpt_model.save_pretrained(f"{OUTPUT_DIR}/sparsegpt_only")

print("✓ SparseGPT model saved")

## Optional: Upload to HuggingFace Hub

In [ ]:
# Upload the best model to HuggingFace
from huggingface_hub import HfApi

repo_name = "your-username/merged-mental-health-counselor"

model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)

print(f"✓ Model uploaded to {repo_name}")

## Memory Monitoring (Optional)

In [ ]:
# Check GPU memory usage
if torch.cuda.is_available():
    print(f"GPU Memory Allocated: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
    print(f"GPU Memory Reserved: {torch.cuda.memory_reserved() / 1e9:.2f} GB")
    
    # Clear cache if needed
    # torch.cuda.empty_cache()